In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
df= pd.read_csv('ax_t1.csv')
df

,Time id,Material,Quantity,Total_Value,Material_Id
0,1,C37-200258,35,392625.71,1708
1,1,C65-200280,45,108333.72,2425
2,1,S02-060397,6,94652.74,6324
3,1,S06-310007,82,88564.92,6587
4,1,S10-130653,32,414240.00,7385
...,...,...,...,...,...
373,38,S06-310007,198,175678.84,6587
374,38,S10-130653,97,740962.30,7385
375,38,S10-130781,132,546171.00,7413
376,38,S12-060214,25,416336.42,14426


In [3]:
import pandas as pd

# Assuming your DataFrame is named df

# Filter the DataFrame for Time ids 37 and 38
filtered_df = df[df['Time id'].isin([37, 38])]

# Display the resulting DataFrame
print(filtered_df)


     Time id          Material  Quantity  Total_Value  Material_Id
357       37  A11-14S-0113-SET         3    181220.99        18176
358       37  A30-14S-0131-SET        13    935101.61        18997
359       37        C37-200258        16    228653.01         1708
360       37        C65-200280        29     96347.63         2425
361       37        S02-060397         4     60536.04         6324
362       37        S02-060529        18    369281.80        11101
363       37        S06-310007       161    115333.28         6587
364       37        S10-130653       121    921052.56         7385
365       37        S10-130781       144    591126.30         7413
366       37        S12-060214        36    607009.96        14426
367       38  A11-14S-0113-SET         8    490059.19        18176
368       38  A30-14S-0131-SET        26   1870716.06        18997
369       38        C37-200258        11    156982.44         1708
370       38        C65-200280        48    157149.14         

In [4]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 1708
Training SimpleRNN model for Material ID: 1708


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
MAE: 0.09561039358377459, MAPE: inf, R-squared: 0.895713657173765
Training GRU model for Material ID: 1708


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
MAE: 0.15565709620714188, MAPE: inf, R-squared: 0.7134094839036311
Training LSTM model for Material ID: 1708


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
MAE: 0.12610366702079776, MAPE: inf, R-squared: 0.7385068513543497
Training LSTM model for Material ID: 22297


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
MAE: 0.14883365094661716, MAPE: inf, R-squared: 0.6429069779595535
   Time id          Material Quantity  Total_Value Material_Id      Model  \
0       37        C37-200258       11    156982.44        1708  SimpleRNN   
1       37        C37-200258       11    156982.44        1708        GRU   
2       37        C37-200258       11    156982.44        1708       LSTM   
3       37        C65-200280       48    157149.14        2425  SimpleRNN   
4       37        C65-200280       48    157149.14        2425        GRU   
5       37        C65-200280       48    157149.14        2425       LSTM   
6       37        S02-060397        2     30268.02        6324  SimpleRNN   
7       37        S02-060397        2     30268.02        6324        GRU   
8       37        S02-060397        2     30268.02        6324       LSTM   
9       37        S06-310007      198    175678.84        6587  SimpleRNN   
10      

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [5]:


# Group the forecast DataFrame by the 'Model' column and calculate the mean MAE and R2 for each group
model_metrics = forecast_df.groupby('Model').agg({'MAE': 'mean', 'R2': 'mean'}).reset_index()

# Print the overall MAE and R2 for each model
print("Overall MAE and R2 for each model:")
print(model_metrics)



Overall MAE and R2 for each model:
       Model       MAE        R2
0        GRU  0.128291  0.764134
1       LSTM  0.156600  0.655209
2  SimpleRNN  0.079706  0.882389


In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 1708

C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Evaluating SimpleRNN model for Material ID: 1708
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
MAPE: 10.58114767074585
Evaluating GRU model for Material ID: 1708


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
MAPE: 0.8101582527160645
Evaluating LSTM model for Material ID: 1708


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
MAPE: 37.72555131178636
Evaluating LSTM model for Material ID: 18997


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
MAPE: 40.50099299504207
Evaluating models for Material ID: 22297
No training or test data available for Material ID: 22297. Skipping evaluation.
   Material_Id      Model        MAPE
0         1708  SimpleRNN   10.581148
1         1708        GRU    0.810158
2         1708       LSTM    1.330876
3         2425  SimpleRNN   49.611716
4         2425        GRU   57.336847
5         2425       LSTM   59.485692
6         6324  SimpleRNN  105.124354
7         6324        GRU  108.037353
8         6324       LSTM  112.239766
9         6587  SimpleRNN    8.885752
10        6587        GRU   11.276169
11        6587       LSTM   13.499669
12        7385  SimpleRNN   53.366316
13        7385        GRU   51.685835
14        7385       LSTM   51.848694
15        7413  SimpleRNN   38.859055
16        7413        GRU   41.065317
17        7413       LSTM   43.115155
18       11101  SimpleRNN   74.908084
19       11101        GRU   72.381801
20       11101    

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [7]:

# Calculate the overall MAPE for each model type
overall_mape = forecast_df.groupby('Model')['MAPE'].mean().reset_index()

# Print the overall MAPE for each model type
print("Overall MAPE for each model type:")
print(overall_mape)

Overall MAPE for each model type:
       Model       MAPE
0        GRU  48.558785
1       LSTM  49.328144
2  SimpleRNN  48.918542


In [8]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 1708
Training SimpleRNN model for Material ID: 1708


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
MAE: 0.0023835855177961694, MAPE: inf, R-squared: 0.9999339306561366
Training GRU model for Material ID: 1708


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
MAE: 0.1178944541857793, MAPE: inf, R-squared: 0.8364628495762942
Training LSTM model for Material ID: 1708


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
MAE: 0.11555283129215244, MAPE: inf, R-squared: 0.7811437908621355
Training LSTM model for Material ID: 22297


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
MAE: 0.15662992656230929, MAPE: inf, R-squared: 0.565775600813877
   Time id          Material Quantity  Total_Value Material_Id      Model  \
0       38        C37-200258       11    156982.44        1708  SimpleRNN   
1       38        C37-200258       11    156982.44        1708        GRU   
2       38        C37-200258       11    156982.44        1708       LSTM   
3       38        C65-200280       48    157149.14        2425  SimpleRNN   
4       38        C65-200280       48    157149.14        2425        GRU   
5       38        C65-200280       48    157149.14        2425       LSTM   
6       38        S02-060397        2     30268.02        6324  SimpleRNN   
7       38        S02-060397        2     30268.02        6324        GRU   
8       38        S02-060397        2     30268.02        6324       LSTM   
9       38        S06-310007      198    175678.84        6587  SimpleRNN   
10      3

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [9]:



# Group the forecast DataFrame by the 'Model' column and calculate the mean MAE and R2 for each group
model_metrics = forecast_df.groupby('Model').agg({'MAE': 'mean', 'R2': 'mean'}).reset_index()

# Print the overall MAE and R2 for each model
print("Overall MAE and R2 for each model:")
print(model_metrics)


Overall MAE and R2 for each model:
       Model       MAE        R2
0        GRU  0.131841  0.729484
1       LSTM  0.150624  0.663648
2  SimpleRNN  0.055816  0.928087


In [10]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 1708
Evaluating SimpleRNN model for Material ID: 1708


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
MAPE: 42.30100458318537
Evaluating GRU model for Material ID: 1708


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
MAPE: 37.28342056274414
Evaluating LSTM model for Material ID: 1708


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
MAPE: 42.85921683678261
Evaluating LSTM model for Material ID: 18997


C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
MAPE: 39.09851954533504
Evaluating models for Material ID: 22297
No training or test data available for Material ID: 22297. Skipping evaluation.
   Material_Id      Model        MAPE
0         1708  SimpleRNN   42.301005
1         1708        GRU   37.283421
2         1708       LSTM   34.189007
3         2425  SimpleRNN   29.390303
4         2425        GRU   25.079632
5         2425       LSTM   16.181087
6         6324  SimpleRNN  106.028080
7         6324        GRU  192.876101
8         6324       LSTM  216.908956
9         6587  SimpleRNN   21.568137
10        6587        GRU   25.148488
11        6587       LSTM   26.854952
12        7385  SimpleRNN   10.530350
13        7385        GRU    0.088871
14        7385       LSTM    4.517066
15        7413  SimpleRNN    6.144946
16        7413        GRU   13.104653
17        7413       LSTM   15.787419
18       11101  SimpleRNN   47.977463
19       11101        GRU   22.927499
20       11101     

C:\Users\sohan\AppData\Local\Temp\ipykernel_22764\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [11]:
# Calculate the overall MAPE for each model type
overall_mape = forecast_df.groupby('Model')['MAPE'].mean().reset_index()

# Print the overall MAPE for each model type
print("Overall MAPE for each model type:")
print(overall_mape)

Overall MAPE for each model type:
       Model       MAPE
0        GRU  43.146558
1       LSTM  42.244627
2  SimpleRNN  41.132548
